# Advent of Code 2024 Day 17 

### Setup

In [3]:
from aocd import get_data, submit

day = 17
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
def parse_data(raw_data):
    lines = raw_data.splitlines()
    
    data = {'instructions': [], 'registers': {}}
    for line in lines[:3]:
        if 'A' in line:
            data['registers']['A'] = int(line.replace('Register A: ', ''))
        
        if 'B' in line:
            data['registers']['B'] = int(line.replace('Register B: ', ''))
            
        if 'C' in line:
            data['registers']['C'] = int(line.replace('Register C: ', ''))
        
    data['instructions'] = [int(x.strip()) for x in lines[-1].replace('Program:' , '').split(',')]
    
    return data
            

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

### Part One!

In [7]:
use_sample_data = False
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data
data

In [9]:
from typing import Dict

class Registers():
    
    def __init__(self, mappings: Dict):
        self._registers = mappings
        
    def __getitem__(self, key):
        return self._registers[key]
    
    def __setitem__(self, key, value):
        self._registers[key] = value
        
    def __repr__(self):
        return str(self._registers)
    
    def __str__(self):
        return str(self._registers)
    
    def __iter__(self):
        return iter(self._registers)
    
    def __len__(self):
        return len(self._registers)
    
    def __contains__(self, key):
        return key in self._registers
    
    def keys(self):
        return self._registers.keys()
    
    def values(self):
        return self._registers.values()
    
    def items(self):
        return self._registers.items()
    
    def copy(self):
        return Registers(self._registers.copy())

In [10]:
from typing import Union, Tuple, Any, List

OperationResult = Tuple[int, Registers, Any]

In [11]:
def get_combo(opcode, registers):
    if opcode <= 3:
        return opcode
    
    elif opcode == 4:
        return registers['A']
    
    elif opcode == 5:
        return registers['B']
    
    elif opcode == 6:
        return registers['C']
    
    else:
        return -1

In [12]:
def get_literal(opcode, registers):
    return opcode

In [13]:
import math

def adv(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    registers['A'] //= 2 ** combo
    
    return (pointer + 2, registers, None)

In [14]:
def bxl(operand:int, registers:Registers, pointer:int) -> OperationResult:
    literal = get_literal(operand, registers)
    registers['B'] ^= literal
    
    return (pointer + 2, registers, None)

In [15]:
def bst(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    registers['B'] = combo % 8
    
    return (pointer + 2, registers, None)

In [16]:
def jnz(operand:int, registers:Registers, pointer:int) -> OperationResult:
    if registers['A']:
        return (get_literal(operand, registers), registers, None)
    
    return (pointer + 2, registers, None)

In [17]:
def bxc(operand:int, registers:Registers, pointer:int) -> OperationResult:
    registers['B'] ^= registers['C']
    
    return (pointer + 2, registers, None)

In [18]:
def out(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    
    return (pointer + 2, registers, combo % 8)

In [19]:
def bdv(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    registers['B'] = registers['A'] // 2 ** combo
    
    return (pointer + 2, registers, None)

In [20]:
def cdv(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    registers['C'] = registers['A'] // 2 ** combo
    
    return (pointer + 2, registers, None)

In [21]:
from typing import Callable

def get_operation(opcode:int) -> Callable:
    if opcode == 0:
        return adv
    
    elif opcode == 1:
        return bxl
    
    elif opcode == 2:
        return bst
    
    elif opcode == 3:
        return jnz
    
    elif opcode == 4:
        return bxc
    
    elif opcode == 5:
        return out
    
    elif opcode == 6:
        return bdv
    
    elif opcode == 7:
        return cdv
    

In [22]:
def perform_instruction(opcode:int, operand:int, registers:Registers, pointer:int) -> OperationResult:
    operation = get_operation(opcode)
    
    return operation(operand, registers, pointer)

In [23]:
def run_program(instructions:List[str], registers:Registers) -> Registers:
    pointer = 0
    
    outputs = []
    while pointer < len(instructions):
        opcode, operand = instructions[pointer:pointer+2]
        
        pointer, registers, output = perform_instruction(opcode, operand, registers, pointer)
        
        if output is not None:
            outputs.append(output)
    
    return (registers, outputs)

In [ ]:
instructions = data['instructions']
starting_registers = Registers(data['registers'])

registers, output = run_program(instructions, starting_registers.copy())

part_a_answer = ",".join([str(x) for x in output])
part_a_answer, use_sample_data

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [49]:
use_sample_data = False
part='b'

In [50]:
sample_data = {
    'registers': {'A': 2024, 'B': 0, 'C': 0},
    'instructions': [0,3,5,4,3,0]
}

In [ ]:
data = sample_data if use_sample_data else test_data

data

In [52]:
import numpy as np

def find_program_copy_locations(instructions:List[int], instruction_idx=None, reg_a=0) -> List[int]:
    instruction_idx
    if instruction_idx < 0:
        return reg_a

    for candidate in range(8):
        val = reg_a * 8 + candidate
        _, output = run_program(instructions, Registers({'A': val, 'B': 0, 'C': 0}))

        if output and output[0] == instructions[instruction_idx]:
            result = find_program_copy_locations(instructions, instruction_idx - 1, val)

            if result is not None:
                return result
            
    return None 
             

In [ ]:
instructions = data['instructions']
starting_registers = Registers(data['registers'])

part_b_answer = find_program_copy_locations(instructions)
part_b_answer

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)